In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycountry

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
from datetime import datetime, timedelta

def generate_birthday():
    # Generate random age following a distribution
    age = random.choices([
        random.randint(20, 25),  # Less frequent
        random.randint(25, 30),  # Most frequent
        random.randint(30, 35),  # More frequent
        random.randint(35, 50)   # Least frequent
    ],weights=[3,5,4,2])[0]
    
    # Calculate corresponding birthdate based on age
    birth_year = datetime.now().year - age
    birth_month = random.randint(1, 12)
    birth_day = random.randint(1, 28)  # Assuming maximum day is 28 for simplicity
    
    birthdate = datetime(birth_year, birth_month, birth_day)
    return birthdate.strftime("%Y-%m-%d")

# Generate a list of birthdays
num_birthdays = 1000  # Adjust as needed
birthdays = [generate_birthday() for _ in range(num_birthdays)]

# Count the occurrences of each age group
age_counts = {
    "20-25": sum(20 <= (datetime.now().year - datetime.strptime(bd, "%Y-%m-%d").year) <= 25 for bd in birthdays),
    "25-35": sum(25 <= (datetime.now().year - datetime.strptime(bd, "%Y-%m-%d").year) <= 35 for bd in birthdays),
    "35-50": sum(35 <= (datetime.now().year - datetime.strptime(bd, "%Y-%m-%d").year) <= 50 for bd in birthdays)
}

print("Age Group Counts:", age_counts)


Age Group Counts: {'20-25': 278, '25-35': 693, '35-50': 186}


In [2]:
import random
from datetime import datetime, timedelta

def generate_birthday():
    # Generate random age following a distribution
    age = random.choices([
        random.randint(20, 25),  # Less frequent
        random.randint(25, 30),  # Most frequent
        random.randint(30, 35),  # More frequent
        random.randint(35, 50)   # Least frequent
    ],weights=[3,5,4,2])[0]
    
    # Calculate corresponding birthdate based on age
    birth_year = datetime.now().year - age
    birth_month = random.randint(1, 12)
    birth_day = random.randint(1, 28)  # Assuming maximum day is 28 for simplicity
    
    birthdate = datetime(birth_year, birth_month, birth_day)
    return birthdate


In [10]:
generate_birthday()

datetime.datetime(1996, 8, 25, 0, 0)

In [3]:
import json
import random
from datetime import datetime, timedelta
from faker import Faker
import pycountry

# Create a Faker object
fake = Faker()

# Get a list of all countries and Extracting names of each country
all_countries = list(pycountry.countries)
countries = [country.alpha_3 for country in all_countries]
# List of possible gender values
genders = ["male", "female"]

# List of possible highestDegree values and weights to create biased data
degree = ["Bachelor's Degree","Master's Degree","Doctorate","Post Doctorate"]
w1=[4,5,2,1]

# List of quiz_status and weights to create biased data
status=["complete","incomplete"]
w2=[5,2]


start_date = datetime(2023, 8, 1)
end_date = datetime(2023, 9, 30)

applicants = []
current_date = start_date

while current_date <= end_date:
    num_applicants = random.randint(25, 50)
    for _ in range(num_applicants):
        dummy_name = fake.name()
        quiz_status= random.choices(status, weights=w2)[0]
        applicant = {
            "_id": len(applicants) + 1,
            "createdAt": (current_date + timedelta(seconds=random.uniform(0,24*60*60))),
            "name": dummy_name,
            "email": f"{dummy_name}@gmail.com",
            "birthday": generate_birthday(),
            "gender": random.choice(genders),
            "highestDegree": random.choices(degree, weights=w1)[0],
            "quizStatus": random.choices(status, weights=w2)[0],
            "country": random.choice(countries)
        }
        if applicant["quizStatus"]=="complete":
            applicant["quizscore"]=random.randint(50, 100)
        applicants.append(applicant)
    
    current_date += timedelta(days=1)


In [14]:
from pymongo import MongoClient
from pprint import PrettyPrinter

In [15]:
pp=PrettyPrinter(indent=2)

In [19]:
client = MongoClient("mongodb+srv://<username>:<password>@mydatabase.wo9j7bt.mongodb.net/?retryWrites=true&w=majority")
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [20]:
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'HR_db', 'sizeOnDisk': 405504},
  {'empty': False, 'name': 'admin', 'sizeOnDisk': 344064},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 18331357184}]


In [22]:
db=client["HR_db"]
pp.pprint(list(db.list_collections()))

[ { 'idIndex': {'key': {'_id': 1}, 'name': '_id_', 'v': 2},
    'info': { 'readOnly': False,
              'uuid': Binary(b'\xec\xb1\xaf\x82\xbbLGE\x9cq\x03\xc1g`\xee\x0b', 4)},
    'name': 'applicants_data',
    'options': {},
    'type': 'collection'}]


In [28]:
records=db["applicants_data"]
records.count_documents({})

0

In [29]:
records.insert_many(applicants)

In [30]:
records.count_documents({})

2266

In [31]:
records.find_one()

{'_id': 1,
 'createdAt': datetime.datetime(2023, 8, 1, 9, 30, 6, 847000),
 'name': 'Rita Brown',
 'email': 'Rita Brown@gmail.com',
 'birthday': datetime.datetime(1998, 3, 14, 0, 0),
 'gender': 'female',
 'highestDegree': 'Post Doctorate',
 'quizStatus': 'incomplete',
 'country': 'LSO'}

In [32]:
# Close the MongoDB connection
client.close()